## We need the lat-lon wind for plotting streamlines and calculating Liapunov components

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs

import cubedsphere as cs
import xesmf as xe

In [2]:
maindir = '/n/home03/zhuangjw/seasasfs/fromPleiades/output_FV3/C384/C384L160/'
ds = cs.open_FV3data(maindir,'winds_daily')
ds

<xarray.Dataset>
Dimensions:  (pfull: 160, phalf: 161, tile: 6, time: 8, x: 384, x_b: 385, y: 384, y_b: 385)
Coordinates:
  * x        (x) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * y        (y) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * pfull    (pfull) float64 2.205 6.781 13.27 19.62 25.92 32.21 38.49 44.76 ...
  * phalf    (phalf) float64 1.0 4.125 10.39 16.65 22.92 29.18 35.44 41.71 ...
  * time     (time) float64 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0
    lon      (tile, y, x) float32 305.097 305.293 305.488 305.685 305.881 ...
    lat      (tile, y, x) float32 -35.2184 -35.3098 -35.4009 -35.4917 ...
  * x_b      (x_b) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * y_b      (y_b) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
    lon_b    (tile, y_b, x_b) float32 305.0 305.195 305.391 305.586 305.783 ...
    lat_b    (tile, y_b, x_b) float32 -35.2644 -35.3561 -35.4474 -35.5385 ...
    area     (tile, y, x) f

In [7]:
ds_out = xe.grid_global(1, 1)
ds_out

<xarray.Dataset>
Dimensions:  (x: 360, x_b: 361, y: 180, y_b: 181)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 -175.5 -174.5 -173.5 ...
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 -89.5 -89.5 -89.5 -89.5 ...
    lon_b    (y_b, x_b) int64 -180 -179 -178 -177 -176 -175 -174 -173 -172 ...
    lat_b    (y_b, x_b) int64 -90 -90 -90 -90 -90 -90 -90 -90 -90 -90 -90 ...
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    *empty*

In [8]:
def C2L(ds_in, ds_out, dr_in):
    
    for k in range(6):
        print(k, end=' ')
        dr_temp = xe.regrid(ds_in.isel(tile=k), ds_out, 
                            dr_in.isel(tile=k), method='conservative')
        if k==0:
            dr_out = dr_temp
        else:
            dr_out += dr_temp
    
    return  dr_out

In [9]:
# load into memory before regridding
dr_u = ds['u700'].load()
dr_v = ds['v700'].load()

In [10]:
# take very long
u_out = C2L(ds, ds_out, dr_u)

0 1 2 3 4 5 

In [11]:
outputdir = '../../outputdata/regridded/'
u_out.to_netcdf(outputdir+'C384L160_u700.nc')

In [12]:
# take very long
v_out = C2L(ds, ds_out, dr_v)

0 1 2 3 4 5 

In [13]:
outputdir = '../../outputdata/regridded/'
v_out.to_netcdf(outputdir+'C384L160_v700.nc')